# 1. Passo a passo

1. Baixar base de dados
2. Criar as amostras com reposição e colocar numa lista
3. Para cada amostra, dividir em treino e teste, selecionando aleatoriamente apenas duas colunas
4. Para cada amostra, treinar o modelo
5. Para cada amostra, criar uma previsões de teste. O dado a ser usado para o teste deve conter apenas as colunas nas quais o modelo foi criado
6. Juntar todas as previsões em um dataframe.
7. Fazer a agregação a partir das frequências

# 2. Descrição

O Random Forest é um processo de junção de modelos para melhorar a eficácia do resultado. Consiste em se criar amostras variadas com repetição em cima do conjunto de dados inicial. Cada amostra deve conter um número reduzido de colunas (geralmente a raiz do total de colunas) e devem ser escolhidas aleatoriamente. Psoteriormente, serão utilizadas para treinar um modelo e cada modelo será utilizado para se tirar previsões. À partir dessas previsões, utilizamos uma ténica de agregação para entender qual o melhor resultado. No caso de variáveis discretas, o melhor resultado é o que tiver maior frequência entre os diferentes modelos. Para variáveis contínuas, o melhor resultado é a média de todos os resultados.

In [1]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = sns.load_dataset("penguins")
df.dropna(inplace=True)
df['species'] = df['species'].map({'Adelie': 0, 'Chinstrap':1, 'Gentoo': 2 })
df['sex'] = df['sex'].map({'Male':0, 'Female':1})
df


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,Torgersen,39.1,18.7,181.0,3750.0,0
1,0,Torgersen,39.5,17.4,186.0,3800.0,1
2,0,Torgersen,40.3,18.0,195.0,3250.0,1
4,0,Torgersen,36.7,19.3,193.0,3450.0,1
5,0,Torgersen,39.3,20.6,190.0,3650.0,0
...,...,...,...,...,...,...,...
338,2,Biscoe,47.2,13.7,214.0,4925.0,1
340,2,Biscoe,46.8,14.3,215.0,4850.0,1
341,2,Biscoe,50.4,15.7,222.0,5750.0,0
342,2,Biscoe,45.2,14.8,212.0,5200.0,1


In [3]:
n_col = math.floor(np.sqrt(len(df.columns)))

In [4]:
amostras = []

for i in range(20):
    amostra = df.sample(len(df), replace=True, random_state=i)
    amostras.append(amostra)
    
amostras

[     species     island  bill_length_mm  bill_depth_mm  flipper_length_mm  \
 178        1      Dream            50.5           18.4              200.0   
 53         0     Biscoe            42.0           19.5              200.0   
 123        0  Torgersen            41.4           18.5              202.0   
 198        1      Dream            50.1           17.9              190.0   
 332        2     Biscoe            43.5           15.2              213.0   
 ..       ...        ...             ...            ...                ...   
 233        2     Biscoe            48.4           14.6              213.0   
 281        2     Biscoe            46.2           14.9              221.0   
 298        2     Biscoe            45.2           13.8              215.0   
 57         0     Biscoe            40.6           18.8              193.0   
 164        1      Dream            47.0           17.3              185.0   
 
      body_mass_g  sex  
 178       3400.0    1  
 53        4

In [5]:

amostras[0].drop(['species', 'island'], axis=1).sample(2, axis='columns')

,body_mass_g,flipper_length_mm
178,3400.0,200.0
53,4050.0,200.0
123,3875.0,202.0
198,3400.0,190.0
332,4650.0,213.0
...,...,...
233,5850.0,213.0
281,5300.0,221.0
298,4750.0,215.0
57,3800.0,193.0


In [6]:
splits = []
for amostra in amostras:
    #print(amostra['island'])
    X = amostra.drop(['species', 'island'], axis=1).sample(2, axis='columns')
    y = amostra['species']
    splits.append(train_test_split(X, y, test_size=0.3))
    
splits

[[     body_mass_g  bill_length_mm
  274       4900.0            46.5
  298       4750.0            45.2
  141       3475.0            40.6
  180       3700.0            46.4
  208       3250.0            45.2
  ..           ...             ...
  93        4450.0            39.6
  224       5400.0            47.6
  255       5400.0            48.4
  263       4750.0            49.6
  167       4050.0            50.5
  
  [233 rows x 2 columns],
       body_mass_g  bill_length_mm
  295       5800.0            48.6
  221       5700.0            50.0
  49        4150.0            42.3
  182       3200.0            40.9
  142       3050.0            32.1
  ..           ...             ...
  50        3500.0            39.6
  205       4050.0            50.7
  46        3425.0            41.1
  204       3600.0            45.7
  265       4900.0            43.6
  
  [100 rows x 2 columns],
  274    2
  298    2
  141    0
  180    1
  208    1
        ..
  93     0
  224    2
  255    2
  2

In [10]:
models=[]

for split in splits:
    model = DecisionTreeClassifier(max_depth=5, min_samples_leaf=20)
    model.fit(split[0], split[2])
    models.append(model)
    
models[0].feature_names_in_

array(['body_mass_g', 'bill_length_mm'], dtype=object)

In [11]:
test_X= df.drop(['species', 'island'], axis=1).sample(1, random_state=241)
test_X

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
157,45.2,17.8,198.0,3950.0,1


In [12]:
predictions = []

for j in range(len(models)):
    features = list(models[j].feature_names_in_)
    new_X = test_X.loc[:,features] 
    y_pred = models[j].predict(new_X)
    predictions.append(y_pred[0])

# for model in models:
#     for split in splits:
#         print(model.
#         y_pred = model.predict(split[1])
#     predictions.append(y_pred[0])
predictions

[1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0]

In [13]:
final_prediction = pd.DataFrame(predictions).value_counts().idxmax()[0]
final_prediction

1